In [1]:
print('hola')

hola


In [2]:
import datacube
acube = datacube.Datacube(app='boku', env='acube')
import datetime
from datetime import timedelta, date
from datacube.utils.cog import write_cog
from pylab import *
import numpy as np
import os
import matplotlib.colors as clr
import itertools
import csv

/app/opt/venv/lib/python3.8/site-packages/datacube/drivers/postgres/_connections.py:81: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  EngineUrl(


In [3]:
def query_data (latitude, longitude, date):
    query = {
        'product': 'B_Sentinel_2',
        'output_crs': 'EPSG:32633',
        'resolution': (-10, 10),
        'lon': longitude,
        'lat': latitude,
        'time': date,
        'measurements': ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12'],
        'cloud_cover_percentage': (0.0, 50.0)
    }
    data = acube.load(**query)
    data_array = np.array(data.to_array(), dtype='float')
    for dataset in acube.find_datasets_lazy(**query):
        geo = dataset.metadata_doc['extent']['coord']['ll']
    return data_array, geo


def image_selection_byCloud(fdate, latitude, longitude, cpercen, mode):
    '''
    fdate = tuple(YYYY, MM, DD)
    latitude = tuple(xx, yy)
    longitude = tuple(xx, yy)
    cpercen = str
    mode = str('before') or str('after')
    '''

    # date range
    sdate = fdate  # start date: flood date
    if mode == 'before':
        edate = date(2015, 7, 1)  # end date: first sentinel 2 image
        delta = sdate - edate  # as timedelta
        time = (str(edate), str(sdate))  # time range for query
    elif mode == 'after':
        now = datetime.datetime.now()
        edate = date(now.year, now.month, now.day)  # end date: today
        delta = edate - sdate  # as timedelta
        time = (str(sdate), str(edate))  # time range for query
    else:
        print('No valid mode')

    # list available dates in acube: they are not ordered
    c_product = 'CLOUDMASK_Sentinel_2'
    c_query = {
        'lat': latitude,
        'lon': longitude,
        'time': time
    }
    acube_dates = []
    for dataset in acube.find_datasets_lazy(product=c_product, **c_query):
        acube_dates.append((dataset.center_time).strftime("%Y-%m-%d"))

    # for loop per available days: see cloud coverage
    for i in range(delta.days + 1):
        if mode == 'before':
            day = (sdate - timedelta(days=i)).strftime("%Y-%m-%d")
        elif mode == 'after':
            day = (sdate + timedelta(days=i)).strftime("%Y-%m-%d")
        print('----------------------------------')
        print('New Date:', day)
        # query cloud data
        if day in acube_dates:
            print(f'day {day} is in acube_dates')
            # print('Cloud data querying...')
            cloud_query = {
                'product': 'CLOUDMASK_Sentinel_2',
                'output_crs': 'EPSG:32633',
                'resolution': (-10, 10),
                'lon': longitude,
                'lat': latitude,
                'time': day,
                'measurements': ['band_1'],
            }
            # select image with cloud coverage condition
            cloud = acube.load(**cloud_query)
            # print('Transform query to array...')
            cloud_array = np.array(cloud.to_array())[0]  # there is only one band
            print('Calculating cloud coverage...')
            if len(np.unique(cloud_array[0])) == 1:
                print('No cloud coverage')
                s2_array, s2_geo = query_data(latitude, longitude, day)
                if not np.any(s2_array) == False:
                    print('End')
                    break
                else:
                    print('sentinel 2 band-arrays full of zeros!')
                    continue
            elif len(np.unique(cloud_array[0])) == 2:
                cloud_coverage = np.count_nonzero(cloud_array[0] == 255) / (
                            cloud_array.shape[1] * cloud_array.shape[2]) * 100
                print('Cloud coverage (%):', cloud_coverage)
                if cloud_coverage < cpercen:
                    print(f'Cloud coverage less than {cpercen}%')
                    s2_array, s2_geo = query_data(latitude, longitude, day)
                    if not np.any(s2_array) == False:
                        print('End')
                        break
                    else:
                        print('sentinel 2 band-arrays full of zeros!')
                        continue
                else:
                    print(f'Cloud coverage more than {cpercen}%')
                    continue
            else:
                print('Error in cloud mask')
        else:
            print(f'day {day} is not in acube_dates')
            continue
    # delete temporal dimension (there is only one image)
    s2_array = s2_array.reshape(s2_array.shape[0], s2_array.shape[2], s2_array.shape[3])
    return day, s2_array, cloud_array, s2_geo

In [4]:
fdate = date(2019, 7, 27) # inserted by user
mask_tif = '/home/boku/jupyter-datacube-docker-singularity/masks/mask_Strobler.tif'
path_out = '/home/boku/jupyter-datacube-docker-singularity/results/'
identifier = f'ejemplo' 
cpercen = 20

In [5]:
# Data after and before event:
longitude, latitude = ((13.492695567475213, 13.52431626547622),(47.70332636412736, 47.720484186054044))
day_b, s2_b, cloud_b, s2_geo_b = image_selection_byCloud (fdate, latitude, longitude, cpercen, 'before')
day_a, s2_a, cloud_a, s2_geo_a = image_selection_byCloud (fdate, latitude, longitude, cpercen, 'after')
print('Acube returns B02, B03, B04, B05, B06, B07, B08, B8A, B11 and B12 for Sentinel 2')
print('Shape sentinel 2 array before event:', s2_b.shape)
print('Shape sentinel 2 array after event:', s2_a.shape)

----------------------------------
New Date: 2019-07-27
day 2019-07-27 is not in acube_dates
----------------------------------
New Date: 2019-07-26
day 2019-07-26 is in acube_dates
Calculating cloud coverage...
No cloud coverage
sentinel 2 band-arrays full of zeros!
----------------------------------
New Date: 2019-07-25
day 2019-07-25 is in acube_dates
Calculating cloud coverage...
No cloud coverage
sentinel 2 band-arrays full of zeros!
----------------------------------
New Date: 2019-07-24
day 2019-07-24 is not in acube_dates
----------------------------------
New Date: 2019-07-23
day 2019-07-23 is in acube_dates
Calculating cloud coverage...
No cloud coverage
sentinel 2 band-arrays full of zeros!
----------------------------------
New Date: 2019-07-22
day 2019-07-22 is not in acube_dates
----------------------------------
New Date: 2019-07-21
day 2019-07-21 is not in acube_dates
----------------------------------
New Date: 2019-07-20
day 2019-07-20 is in acube_dates
Calculating cl